<a href="https://colab.research.google.com/github/lookshines/Google_App_Scipt_Codes/blob/main/School_Leader_Output_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import library**

In [ ]:
import pandas as pd
import numpy as np
import gspread

Mount Drive

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


**Import Excel Files**

In [ ]:
school_info = pd.read_excel('/content/drive/MyDrive/NG-Regional School Inspection/Group Nigeria/Kobo Audit Input Folder /School Leader/Output_Template/GROUPNG_School_Leader_Audit_Kobo_Output.xlsx', sheet_name = 'GROUPNG_School Leader Audit')

In [ ]:
Incorrect_ht_record = pd.read_excel('/content/drive/MyDrive/NG-Regional School Inspection/Group Nigeria/Kobo Audit Input Folder /School Leader/Output_Template/GROUPNG_School_Leader_Audit_Kobo_Output.xlsx', sheet_name = 'confirm_incorrect_ht_records...')

In [ ]:
headteacher_attendance = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Consolidated_Headteacher_Attendance.csv')

In [ ]:
headteacher_attendance = headteacher_attendance.drop_duplicates(subset = 'date_teacher_id')

**Additional Column to school info**

In [ ]:
school_info['start_date'] = pd.to_datetime(school_info['starttime']).dt.date

In [ ]:
school_info['end_date'] = pd.to_datetime(school_info['endtime']).dt.date

In [ ]:
school_info['date_validity_check'] = school_info['start_date'] == school_info['end_date']

Pull Data into school Info

Creating unique indentifier in the incorrect ht record

In [ ]:
Incorrect_ht_record['ht_id_correct'] = Incorrect_ht_record.apply(lambda x: str(x['_parent_index']) + 'The employee ID needs correcting / updating' if x['confirm_incorrect_ht_records_index'] == 'The employee ID needs correcting / updating' else '', axis=1)

In [ ]:
Incorrect_ht_record['ht_name_correct'] = Incorrect_ht_record.apply(lambda x: str(x['_parent_index']) + 'The name needs correcting / updating' if x['confirm_incorrect_ht_records_index'] == 'The name needs correcting / updating' else '', axis=1)

In [ ]:
Incorrect_ht_record['ht_phone_correct'] = Incorrect_ht_record.apply(lambda x: str(x['_parent_index']) + 'The phone number needs correcting / updating' if x['confirm_incorrect_ht_records_index'] == 'The phone number needs correcting / updating' else '', axis=1)

Creating unique indentifier in the school info

In [ ]:
school_info['ht_id_correct'] = school_info['_index'].astype(str) + 'The employee ID needs correcting / updating'

In [ ]:
school_info['ht_name_correct'] = school_info['_index'].astype(str) + 'The name needs correcting / updating'

In [ ]:
school_info['ht_phone_correct'] = school_info['_index'].astype(str) + 'The phone number needs correcting / updating'

Clean Up Attendance Data

In [ ]:
headteacher_attendance['date_school'] = headteacher_attendance['ActivityDate'] + headteacher_attendance['AcademyCode']

In [ ]:
headteacher_attendance.copy = headteacher_attendance.drop_duplicates(subset = 'date_school', keep = False)

Bring data from incorrec ht record into school info

In [ ]:
school_info = school_info.merge(Incorrect_ht_record[['ht_id_correct', 'Please enter the correct  **"${confirm_incorrect_ht_records_index}"**']], how = 'left', on = 'ht_id_correct')

In [ ]:
school_info = school_info.merge(Incorrect_ht_record[['ht_name_correct', 'Please enter the correct  **"${confirm_incorrect_ht_records_index}"**']], how = 'left', on = 'ht_name_correct')

In [ ]:
school_info = school_info.merge(Incorrect_ht_record[['ht_phone_correct', 'Please enter the correct  **"${confirm_incorrect_ht_records_index}"**']], how = 'left', on = 'ht_phone_correct')

In [ ]:
school_info.rename({'Please enter the correct  **"${confirm_incorrect_ht_records_index}"**_x' : 'Correct_ht_id', 'Please enter the correct  **"${confirm_incorrect_ht_records_index}"**_y' : 'correct_ht_name', 'Please enter the correct  **"${confirm_incorrect_ht_records_index}"**' : 'correct_ht_phonenumber'}, axis = 1, inplace = True)

Pulling Headteacher details from the Attendance file

In [ ]:
school_info['date_school'] = school_info['start_date'].astype(str) + school_info['School name']

In [ ]:
school_info = school_info.merge(headteacher_attendance.copy[['Teacher_ID', 'TeacherName', 'date_school']], how = 'left', on = 'date_school')

In [ ]:
school_info.rename({'Teacher_ID' : 'Headteacher ID_RS', 'TeacherName' : 'Headteacher Name_RS'}, axis = 1, inplace = True)

In [ ]:
school_info['Final_Headteacher_ID'] = school_info.apply(lambda x: x['Correct_ht_id'] if x['Select which items, if any, need correcting/The employee ID needs correcting / updating'] == 1
 else (x['Headeacher employee id'] if pd.notnull(x['Headeacher employee id'])  else x['Headteacher ID_RS']), axis = 1)

Pulling Attendance from the attendance sheet

In [ ]:
school_info['date_headteacher_ID'] = school_info.apply(lambda x: str(x['start_date']) + str(x['Final_Headteacher_ID']), axis = 1)

In [ ]:
school_info = school_info.merge(headteacher_attendance[['IS_PRESENT', 'date_teacher_id']], how = 'left', left_on = 'date_headteacher_ID', right_on = 'date_teacher_id')

In [ ]:
school_info.drop(['date_teacher_id', 'date_headteacher_ID', 'ht_id_correct', 'ht_name_correct', 'ht_phone_correct'], axis = 'columns', inplace = True)

In [ ]:
Incorrect_ht_record.drop(['ht_id_correct', 'ht_name_correct', 'ht_phone_correct'], axis = 'columns', inplace = True)

**Preparation for Export**

In [ ]:
export_index = 1

In [ ]:
school_info = school_info[(school_info['_index'] >= export_index)]

In [ ]:
Incorrect_ht_record = Incorrect_ht_record[(Incorrect_ht_record['_parent_index'] >= export_index)]

**Prepartion of Google Sheet**

In [ ]:
from google.colab import auth

In [ ]:
auth.authenticate_user()

In [ ]:
from google.auth import default

In [ ]:
creds, _ = default()

In [ ]:
gc  = gspread.authorize(creds)

In [ ]:
sheet_id = '1SHL5MeKkL166hP9veLURMjLWpbkEXU-AYBzVgvyjcdU'

In [ ]:
sh = gc.open_by_key(sheet_id)

**Get Varous Worksheet**

In [ ]:
school_info_worksheet = sh.worksheet('School_Level_Observation')

In [ ]:
incorrect_ht_record_worksheet = sh.worksheet('confirm_incorrect_ht_records')

**Export to google sheet**

In [ ]:
from gspread_dataframe import set_with_dataframe

In [ ]:
set_with_dataframe(school_info_worksheet, school_info, row = 1, col = 1)

In [ ]:
set_with_dataframe(incorrect_ht_record_worksheet, Incorrect_ht_record, row = 1, col = 1)